## Returns

In [1]:
import pandas as pd

close = pd.DataFrame(
    {
        'ABC': [1, 5, 3, 6, 2],
        'EFG': [12, 51, 43, 56, 22],
        'XYZ': [35, 36, 36, 36, 37],},
    pd.date_range('10/01/2018', periods=5, freq='D'))
close

,ABC,EFG,XYZ
2018-10-01,1,12,35
2018-10-02,5,51,36
2018-10-03,3,43,36
2018-10-04,6,56,36
2018-10-05,2,22,37


In [6]:
import pandas as pd


def calculate_returns(close):
    """
    Compute returns for each ticker and date in close.
    
    Parameters
    ----------
    close : DataFrame
        Close prices for each ticker and date
    
    Returns
    -------
    returns : DataFrame
        Returns for each ticker and date
    """
    ret = (close - close.shift(1))/close.shift(1)
    
    return ret 

calculate_returns(close)



,ABC,EFG,XYZ
2018-10-01,NaN,NaN,NaN
2018-10-02,4.000000,3.250000,0.028571
2018-10-03,-0.400000,-0.156863,0.000000
2018-10-04,1.000000,0.302326,0.000000
2018-10-05,-0.666667,-0.607143,0.027778


## Pandas resample

In [2]:
import numpy as np
import pandas as pd

dates = pd.date_range('10/10/2018', periods=11, freq='D')
close_prices = np.arange(len(dates))

close = pd.Series(close_prices, dates)
close


2018-10-10     0
2018-10-11     1
2018-10-12     2
2018-10-13     3
2018-10-14     4
2018-10-15     5
2018-10-16     6
2018-10-17     7
2018-10-18     8
2018-10-19     9
2018-10-20    10
Freq: D, dtype: int64

In [3]:
# break the close into chucks of size 3
close.resample('3D')

close.resample('3D').first()


2018-10-10    0
2018-10-13    3
2018-10-16    6
2018-10-19    9
Freq: 3D, dtype: int64

In [4]:
close.iloc[::3]


2018-10-10    0
2018-10-13    3
2018-10-16    6
2018-10-19    9
Freq: 3D, dtype: int64

In [5]:
try:
    # Attempt resample on a series without a time index
    pd.Series(close_prices).resample('W')
except TypeError:
    print('It threw a TypeError.')
else:
    print('It worked.')

It threw a TypeError.


One of the resampling tasks it can help with is resampling on periods, like weeks. Let's resample close from it's days frequency to weeks. We'll use the "W" offset allies, which stands for Weeks.

In [9]:
pd.DataFrame({
    'days': close,
    'weeks': close.resample('W').first()})

,days,weeks
2018-10-10,0.0,NaN
2018-10-11,1.0,NaN
2018-10-12,2.0,NaN
2018-10-13,3.0,NaN
2018-10-14,4.0,4.0
2018-10-15,5.0,NaN
2018-10-16,6.0,NaN
2018-10-17,7.0,NaN
2018-10-18,8.0,NaN
2018-10-19,9.0,NaN


The weeks offset considers the start of a week on a Monday. Since 2018-10-10 is a Wednesday, the first group only looks at the first 5 items.

## OLHC


In [7]:
# get olhc from close resampled weekly
close.resample('W').ohlc()


,open,high,low,close
2018-10-14,0,4,0,4
2018-10-21,5,10,5,10


This is wrong as we resample already sampled data. 
We're getting the OHLC from close data. If we want OHLC data from already resampled data, we should resample the first price from the open data, resample the highest price from the high data, etc..

To get the weekly closing prices from close, you can use the Resampler.last function.


In [8]:


close.resample('W').last()


2018-10-14     4
2018-10-21    10
Freq: W-SUN, dtype: int64

In [11]:
def days_to_weeks(open_prices, high_prices, low_prices, close_prices):
    """Converts daily OHLC prices to weekly OHLC prices.
    
    Parameters
    ----------
    open_prices : DataFrame
        Daily open prices for each ticker and date
    high_prices : DataFrame
        Daily high prices for each ticker and date
    low_prices : DataFrame
        Daily low prices for each ticker and date
    close_prices : DataFrame
        Daily close prices for each ticker and date

    Returns
    -------
    open_prices_weekly : DataFrame
        Weekly open prices for each ticker and date
    high_prices_weekly : DataFrame
        Weekly high prices for each ticker and date
    low_prices_weekly : DataFrame
        Weekly low prices for each ticker and date
    close_prices_weekly : DataFrame
        Weekly close prices for each ticker and date
    """
    # open prices weekly
    open_prices_weekly = open_prices.resample('W').first()
    # close prices weekly
    close_prices_weekly = close_prices.resample('W').last()
    # high prices weekly
    high_prices_weekly = high_prices.resample('W').max()
    # low prices weekly
    low_prices_weekly = low_prices.resample('W').min()
    
    return open_prices_weekly, high_prices_weekly, low_prices_weekly, close_prices_weekly


olhc = close.resample('D').ohlc()

open_prices = olhc.open
high_prices = olhc.high
low_prices = olhc.low
close_prices = olhc.close


days_to_weeks(open_prices, high_prices, low_prices, close_prices)

(2018-10-14    0
 2018-10-21    5
 Freq: W-SUN, Name: open, dtype: int64,
 2018-10-14     4
 2018-10-21    10
 Freq: W-SUN, Name: high, dtype: int64,
 2018-10-14    0
 2018-10-21    5
 Freq: W-SUN, Name: low, dtype: int64,
 2018-10-14     4
 2018-10-21    10
 Freq: W-SUN, Name: close, dtype: int64)

## Momentum 

In [39]:
def generate_positions(prices):
    """
    Generate the following signals:
     - Long 30 share of stock when the price is above 50 dollars
     - Short 10 shares when it's below 20 dollars
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    
    Returns
    -------
    final_positions : DataFrame
        Final positions for each ticker and date
    """

    # convert the signal to int
    signal_long = (prices > 50)
    # convert the signal to int
    signal_long = signal_long.astype(np.int64)
    
    signal_short = (prices < 20)
    signal_short = signal_short.astype(np.int64)
    
    pos_long = 30 * signal_long
    pos_short = -10 * signal_short
    
    return pos_long + pos_short
# generate prices for a stock which goes between 0 and 100
prices = pd.DataFrame(
    {
        'ABC': np.random.randint(0, 100, 10),
        'EFG': np.random.randint(0, 100, 10),
        'XYZ': np.random.randint(0, 100, 10)
        },
    pd.date_range('10/01/2018', periods=10, freq='D'))

positions = generate_positions(prices)

# rename tickers in positions to append pos_ and add positions to prices

positions.columns = [f'pos_{c}' for c in positions.columns]

prices = pd.concat([prices, positions], axis=1)

prices

,ABC,EFG,XYZ,pos_ABC,pos_EFG,pos_XYZ
2018-10-01,62,36,4,30,0,-10
2018-10-02,57,82,2,30,30,-10
2018-10-03,47,95,40,0,30,0
2018-10-04,27,52,3,0,30,-10
2018-10-05,0,25,88,-10,0,30
2018-10-06,26,36,66,0,0,30
2018-10-07,95,79,19,30,30,-10
2018-10-08,53,55,8,30,30,-10
2018-10-09,83,11,33,30,-10,0
2018-10-10,35,80,83,0,30,30
